In [1]:
## customer heirachy to classify the umbrella customers and the sum customers




In [1]:
#import Pkg; Pkg.add("AbstractTrees")
using AbstractTrees

In [2]:
##obtain the roots 

using ODBC
conn = ODBC.Connection("ricohnz-db2","Ajefferi","Ca\$h1998")

┌ Info: 01000: [Microsoft][ODBC SQL Server Driver][SQL Server]Changed database context to 'master'.01000: [Microsoft][ODBC SQL Server Driver][SQL Server]Changed language setting to us_english.
└ @ ODBC.API C:\Users\Ajefferi\.julia\packages\ODBC\2MEHX\src\API.jl:112


ODBC.Connection(ricohnz-db2)

In [3]:
query = """

SELECT Account,Customer,Parent_Account,Parent_Customer
  FROM [Ricoh_SAS].[pbi].[Customer_Master] cm 


  
"""

"\nSELECT Account,Customer,Parent_Account,Parent_Customer\n  FROM [Ricoh_SAS].[pbi].[Customer_Master] cm \n\n\n  \n"

In [4]:
using DataFrames

global df = DBInterface.execute(conn,query) |> DataFrame



,Account,Customer,Parent_Account,Parent_Customer
,String,String?,String?,String?
1,10050,NORMAN DISNEY & YOUNG,10050,NORMAN DISNEY & YOUNG
2,10051,NORWEST ADVERTISING LTD,10051,NORWEST ADVERTISING LTD
3,10055,JOANNE ANDERSON,10055,JOANNE ANDERSON
4,10062,BUDGET ****USE 19823*********,10062,BUDGET ****USE 19823*********
5,10066,APPLEBY AUTOMOTIVE ENGINEERING,10066,APPLEBY AUTOMOTIVE ENGINEERING
6,10089,BUDGET **USE 19823**,10089,BUDGET **USE 19823**
7,10097,BUDGET ***USE 19823********,10097,BUDGET ***USE 19823********
8,10098,BUDGET ****USE 19823*******,10098,BUDGET ****USE 19823*******
9,10101,EMPLOYERS & MANUFACTURERS ASSOC,10101,EMPLOYERS & MANUFACTURERS ASSOC


In [6]:
#import Pkg; Pkg.add("LeftChildRightSiblingTrees")
#using LeftChildRightSiblingTrees

In [5]:
parents = df[df.Account .== df.Parent_Account,:];
head(parents)

,Account,Customer,Parent_Account,Parent_Customer
,String,String?,String?,String?
1,10050,NORMAN DISNEY & YOUNG,10050,NORMAN DISNEY & YOUNG
2,10051,NORWEST ADVERTISING LTD,10051,NORWEST ADVERTISING LTD
3,10055,JOANNE ANDERSON,10055,JOANNE ANDERSON
4,10062,BUDGET ****USE 19823*********,10062,BUDGET ****USE 19823*********
5,10066,APPLEBY AUTOMOTIVE ENGINEERING,10066,APPLEBY AUTOMOTIVE ENGINEERING
6,10089,BUDGET **USE 19823**,10089,BUDGET **USE 19823**


In [6]:
children = df[df.Account .!= df.Parent_Account,:];
head(children)

,Account,Customer,Parent_Account,Parent_Customer
,String,String?,String?,String?
1,10236,FORD MOTOR COMPANY OF NEW ZEALAND,71221,AUCKLAND AUTO COLLECTION LTD
2,10245,METLIFECARE WAIRARAPA LIMITED,10269,METLIFECARE REMUERA LTD
3,1999,WAKEFIELD METALS LIMITED,71726,MICO WAKEFIELD A DIVISIN OF CRANE
4,1967,UDC FINANCE LIMITED,82600,ANZ BANK LTD
5,19825,TETRA PAK (CPS) LIMITED,74604,TETRA PAK (NEW ZEALAND) LTD
6,25369,FOODSTUFFS WGTN COOP SOC LTD,62959,FOODSTUFFS (SI) LIMITED


In [7]:

mutable struct Account5
    isParent::Bool
    Account::String
    Name::String
    Children::Vector
end

In [8]:
function build_roots(df)
    roots = []
    parents = df[df.Account .== df.Parent_Account,:];
    for parent in eachrow(parents)
        root = Account5(true,parent.Account,parent.Customer,[])
        push!(roots,root)
        
    end
    return roots
    
    
end

build_roots (generic function with 1 method)

In [9]:


function build_tree(df)
    roots = build_roots(df)
    
    add_children(roots)
        
    
end



build_tree (generic function with 1 method)

In [10]:
function add_children(roots)
    tree = []
    for root in roots
            
            #println("isparent")
            
            childlist = []
            for child in eachrow(children)
                #@show child.Parent_Account,root.Account
                if child.Parent_Account == root.Account
                    #println("found one at $root")
                
                    childAcc = Account5(true,child.Account,child.Customer,[])
                    push!(childlist,childAcc)
                    
                    add_children(childlist)

        
                end
                
                
            end             
            root.Children = childlist
            if root.Children != []
                 #@show root
            end
            #@show root
            if length(childlist)== 0
                #println("notParent")
                root.isParent = false
            
            end    
            #println("nextroot")
                     
    push!(tree,root)
    end
       
    return tree
    
end
    


T = build_tree(df)

30604-element Array{Any,1}:
 Account5(false, "10050", "NORMAN DISNEY & YOUNG", Any[])
 Account5(false, "10051", "NORWEST ADVERTISING LTD", Any[])
 Account5(false, "10055", "JOANNE ANDERSON", Any[])
 Account5(false, "10062", "BUDGET ****USE 19823*********", Any[])
 Account5(false, "10066", "APPLEBY AUTOMOTIVE ENGINEERING", Any[])
 Account5(false, "10089", "BUDGET **USE 19823**", Any[])
 Account5(false, "10097", "BUDGET ***USE 19823********", Any[])
 Account5(false, "10098", "BUDGET ****USE 19823*******", Any[])
 Account5(false, "10101", "EMPLOYERS & MANUFACTURERS ASSOC", Any[])
 Account5(false, "10103", "AUCKLAND COPIERS***IN RECEIVERSHIP*", Any[])
 Account5(false, "10104", "HOLIDAY SHOPPE", Any[])
 Account5(false, "10106", "AUCKLAND DRAPE CO LIMITED", Any[])
 Account5(false, "10114", "INTERNATIONAL MANAGEMENT GROUP", Any[])
 ⋮
 Account5(false, "33320", "DW DENTICE LTD", Any[])
 Account5(false, "33334", "DESTINATION WAIRARAPA INC", Any[])
 Account5(false, "33339", "HOROWHENUA ADULT LITE

In [11]:
function displayInformation(T)
    
    for parentAccount in T
        numSubs = length(parentAccount.Children)
        
        println("Account: $(parentAccount.Name), No: $(parentAccount.Account)")
        println("Number of Sub Accounts: $numSubs")
        for subAccount in parentAccount.Children
            println("Account: No$(subAccount.Name), No: $(subAccount.Account)")
            if length(subAccount.Children) > 0
                println("depth >2")
            end
            
        end
        println("\n\n")
            
        
        

        
    
    
        
    end
end
    


    

displayInformation (generic function with 1 method)

In [31]:

## query the subaccount of each parent and then remove those sub accounts from our dataframe 

function queryParents(T)
    io = open("C:/Users/Ajefferi/Documents/ChildCustomers.txt", "w")
    for parentAccount in T
       childlist = parentAccount.Children
       
        
        
        if length(childlist) > 0 
            accounts = removeFromDf(childlist) 
            [write(io,"$account\n") for account in accounts]
        end
    
        
        
        
    end
    
    
    
    close(io)
    
    
end

queryParents (generic function with 1 method)

In [32]:
function removeFromDf(childs)
    accounts = []
   for subAccount in  childs
            
            if length(subAccount.Children) > 0
                #println("depth >2: need to run again")
                
                removeFromDf(subAccount.Children)
            
            else
                name = subAccount.Name
                account = subAccount.Account
                push!(accounts,account)
                ##write the account to a file 
                ##write the naame to a file
            
            
            end
            
        end
    
    return accounts
end

removeFromDf (generic function with 1 method)

In [33]:
newdf = deepcopy(df)


newParents = newdf[newdf.Account .== newdf.Parent_Account,1:2];



newParents[:numSubAccounts] = [length(t.Children) for t in T];

In [34]:
newParents;

In [35]:
queryParents(T)

In [ ]:
##write the names of all the parent accounts to a file

